In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [10]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
TARGET_SIZE = 128
BATCH_SIZE = 8
EPOCHS = 10
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/CassavaLeaves"

In [12]:
train_labels = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))

In [13]:
train_labels.label = train_labels.label.astype('str')

generator_params = {
    "dataframe": train_labels,
    "directory": os.path.join(ROOT_DIR, "train_images"),
    "x_col": "image_id",
    "y_col": "label",
    "target_size": (TARGET_SIZE, TARGET_SIZE),
    "batch_size": BATCH_SIZE,
    "class_mode": "sparse"
}

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    shear_range = 0.1,
    height_shift_range = 0.1,
    width_shift_range = 0.1
).flow_from_dataframe(
    subset = "training",
    **generator_params
)

validation_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2
).flow_from_dataframe(
    subset = "validation",
    **generator_params
)

Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.


In [14]:
def createModel():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(TARGET_SIZE, TARGET_SIZE, 3)),
      tf.keras.applications.EfficientNetB0(include_top=False, weights=None),
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(5, activation="softmax")
  ])
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                loss = "sparse_categorical_crossentropy",
                metrics = ["acc"])
  return model

In [15]:
model = createModel()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 4055976 (15.47 MB)
Trainable params: 4013953 (15.31 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


In [16]:
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE

history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS
)

Epoch 1/10
2139/2139 [==============================] - 1063s 481ms/step - loss: 1.1471 - acc: 0.6135 - val_loss: 1.5148 - val_acc: 0.4111
Epoch 2/10
2139/2139 [==============================] - 301s 140ms/step - loss: 1.0231 - acc: 0.6311 - val_loss: 0.9622 - val_acc: 0.6480
Epoch 3/10
2139/2139 [==============================] - 292s 136ms/step - loss: 0.9865 - acc: 0.6411 - val_loss: 0.9938 - val_acc: 0.6546
Epoch 4/10
2139/2139 [==============================] - 277s 130ms/step - loss: 0.9607 - acc: 0.6487 - val_loss: 1.1948 - val_acc: 0.5644
Epoch 5/10
2139/2139 [==============================] - 277s 129ms/step - loss: 0.9314 - acc: 0.6547 - val_loss: 0.8558 - val_acc: 0.6752
Epoch 6/10
2139/2139 [==============================] - 285s 133ms/step - loss: 0.8774 - acc: 0.6722 - val_loss: 0.8450 - val_acc: 0.6775
Epoch 7/10
2139/2139 [==============================] - 274s 128ms/step - loss: 0.8438 - acc: 0.6855 - val_loss: 0.7647 - val_acc: 0.7102
Epoch 8/10
2139/2139 [===========